In [1]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to avoid too many logging messages
import pandas as pd, numpy as np, random, shutil
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
import sklearn
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow_probability as tfp
import wandb
import yaml

#from IPython import display as ipd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
os.environ["WANDB_NOTEBOOK_NAME"] = "RSNA2023ATDEfficientnet"

wandb.login()

True

### Configs

In [5]:
class CFG:
    wandb         = True

    '''
    competition   = 'rsna-atd' 
    _wandb_kernel = 'm-noda'
    debug         = False
    comment       = 'EfficientNetV1B1-512x512-lr2-vflip'
    exp_name      = 'baseline-v5: ds-v3 + multi_head' 
    '''
    
    # use verbose=0 for silent, vebose=1 for interactive,
    verbose      = 0
    display_plot = True

    # device
    #device = "TPU-VM" #or "GPU"

    model_name = 'EfficientNet'

    seed = 123

    folds = 5
    
    # which folds to train
    selected_folds = [0, 1]

    # size of the image
    img_size = [512, 512]

    batch_size = 8
    epochs = 12

    loss      = 'BCE & CCE'
    
    optimizer = 'Adam'

    augment   = True

    # scale-shift-rotate-shear
    transform = 0.90  # transform prob
    fill_mode = 'constant'
    rot    = 2.0
    shr    = 2.0
    hzoom  = 50.0
    wzoom  = 50.0
    hshift = 10.0
    wshift = 10.0

    # flip
    hflip = True
    vflip = True

    # clip
    clip = False

    # lr-scheduler
    scheduler   = 'exp' # cosine

    # dropout
    drop_prob   = 0.6
    drop_cnt    = 5
    drop_size   = 0.05
    
    # cut-mix-up
    mixup_prob = 0.0
    mixup_alpha = 0.5
    cutmix_prob = 0.0
    cutmix_alpha = 2.5

    # pixel-augment
    pixel_aug = 0.90  # prob of pixel_aug
    sat  = [0.7, 1.3]
    cont = [0.8, 1.2]
    bri  = 0.15
    hue  = 0.05

    # test-time augs
    tta = 1
    
    # target column
    target_col  = [ "bowel_injury", "extravasation_injury", "kidney_healthy", "kidney_low",
                   "kidney_high", "liver_healthy", "liver_low", "liver_high",
                   "spleen_healthy", "spleen_low", "spleen_high"] # not using "bowel_healthy" & "extravasation_healthy"


### Seeding for reproducibility

In [6]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    tf.random.set_seed(SEED)
    print('seeding done!!!')
seeding(CFG.seed)

seeding done!!!


### Prepare train data, test data

In [7]:

# make dataframes
train_df = pd.read_csv("./train_df.csv")
train_paths = glob('./train_png/*_*_*.png')
train_df['image_path'] = f'./train_png/' + \
                            train_df.patient_id.astype(str) + '_' + \
                            train_df.series_id.astype(str) + '_' + \
                            train_df.img_number.astype(str).str.zfill(3) + '.png'

test_paths = glob('./test_png/*_*_*.png')
test_df = pd.DataFrame(test_paths, columns=["image_path"])
test_df['patient_id'] = test_df.image_path.map(lambda x: x.split('_')[-3].replace('png/','')).astype(int)
test_df['series_id'] = test_df.image_path.map(lambda x: x.split('_')[-2]).astype(int)
test_df['img_number'] = test_df.image_path.map(lambda x: x.split('_')[-1].replace('.png','')).astype(int)

In [10]:
tf.io.gfile.exists(train_df.image_path.iloc[0]), tf.io.gfile.exists(test_df.image_path.iloc[0])

(True, True)

### Data split into train and valid

In [16]:
train_df['stratify'] = ''

for col in CFG.target_col:
    train_df['stratify'] += train_df[col].astype(str)

train_df = train_df.reset_index(drop=True)

skf = StratifiedGroupKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.seed)

for  fold , (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['stratify'], train_df['patient_id'])):
    train_df.loc[val_idx, 'fold'] = fold

pd.set_option('display.max_rows', 100)
display(train_df.groupby(['fold', 'patient_id']).size())

fold  patient_id
0.0   96            130
      318           134
      384           110
      470           103
      496           133
                   ... 
4.0   65289         147
      65326         130
      65438         251
      65450         132
      65495         149
Length: 3147, dtype: int64

### Data Augmentation

In [ ]:
def random_int(shape=[], minval=0, maxval=1):
    return tf.random.uniform(shape=shape, minval=minval, maxval=maxval, dtype=tf.int32)

def random_float(shape=[], minval=0.0, maxval=1.0):
    return tf.random.uniform(shape=shape, minval=minval, maxval=maxval, dtype=tf.float32)

def get_matrix(shear, height_zoom, width_zoom, height_shift, width_shift)
    
    # Degrees to radians
    shear = math.pi * shear / 180

    def get_3x3_mat(list):
        return tf.reshape(tf.concat([list], axis=0), [3, 3])
    
    one = tf.constant([1], dtype='float32')
    zero = tf.constant([0], dtype='float32')